In [ ]:
# Cell 1: Imports
import os
import glob
import IPython.display as ipd
from src.trainer import run_training
from src.config import PROCESSED_DIR

def play_latest_results(output_root="/content/output"):
    exp_folders = sorted(glob.glob(os.path.join(output_root, "kin_vits_production-*")), key=os.path.getmtime)
    if not exp_folders:
        print("No training output found yet.")
        return

    latest_exp = exp_folders[-1]
    wavs = sorted(glob.glob(os.path.join(latest_exp, "*.wav")), key=os.path.getmtime, reverse=True)
    
    if not wavs:
        print(f"No WAV files found in {latest_exp} yet. Wait for the first eval step.")
        return

    print(f"🎵 Playing latest samples from: {os.path.basename(latest_exp)}")
    for wav in wavs[:5]:
        print(f"🔈 {os.path.basename(wav)}")
        ipd.display(ipd.Audio(wav))

try:
    run_training(epochs=30)
except KeyboardInterrupt:
    print("Training interrupted. You can still play samples generated so far.")
play_latest_results()